In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [1]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Loading Raw Data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:, 0:27, 0:27]
x_test = x_test[:, 0:27, 0:27]

In [3]:
x_train_flatten = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/255.0
x_test_flatten = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255.0

In [4]:
print(x_train_flatten.shape, y_train.shape)
print(x_test_flatten.shape, y_test.shape)

(60000, 729) (60000,)
(10000, 729) (10000,)


In [5]:
x_train_0 = x_train_flatten[y_train == 0]
x_train_1 = x_train_flatten[y_train == 1]
x_train_2 = x_train_flatten[y_train == 2]
x_train_3 = x_train_flatten[y_train == 3]
x_train_4 = x_train_flatten[y_train == 4]
x_train_5 = x_train_flatten[y_train == 5]
x_train_6 = x_train_flatten[y_train == 6]
x_train_7 = x_train_flatten[y_train == 7]
x_train_8 = x_train_flatten[y_train == 8]
x_train_9 = x_train_flatten[y_train == 9]

x_train_list = [x_train_0, x_train_1, x_train_2, x_train_3, x_train_4, x_train_5, x_train_6, x_train_7, x_train_8, x_train_9]

print(x_train_0.shape)
print(x_train_1.shape)
print(x_train_2.shape)
print(x_train_3.shape)
print(x_train_4.shape)
print(x_train_5.shape)
print(x_train_6.shape)
print(x_train_7.shape)
print(x_train_8.shape)
print(x_train_9.shape)

(5923, 729)
(6742, 729)
(5958, 729)
(6131, 729)
(5842, 729)
(5421, 729)
(5918, 729)
(6265, 729)
(5851, 729)
(5949, 729)


In [6]:
x_test_0 = x_test_flatten[y_test == 0]
x_test_1 = x_test_flatten[y_test == 1]
x_test_2 = x_test_flatten[y_test == 2]
x_test_3 = x_test_flatten[y_test == 3]
x_test_4 = x_test_flatten[y_test == 4]
x_test_5 = x_test_flatten[y_test == 5]
x_test_6 = x_test_flatten[y_test == 6]
x_test_7 = x_test_flatten[y_test == 7]
x_test_8 = x_test_flatten[y_test == 8]
x_test_9 = x_test_flatten[y_test == 9]

x_test_list = [x_test_0, x_test_1, x_test_2, x_test_3, x_test_4, x_test_5, x_test_6, x_test_7, x_test_8, x_test_9]

print(x_test_0.shape)
print(x_test_1.shape)
print(x_test_2.shape)
print(x_test_3.shape)
print(x_test_4.shape)
print(x_test_5.shape)
print(x_test_6.shape)
print(x_test_7.shape)
print(x_test_8.shape)
print(x_test_9.shape)

(980, 729)
(1135, 729)
(1032, 729)
(1010, 729)
(982, 729)
(892, 729)
(958, 729)
(1028, 729)
(974, 729)
(1009, 729)


# Selecting the dataset

Output: X_train, Y_train, X_test, Y_test

In [42]:
n_train_sample_per_class = 200
n_class = 2

X_train = x_train_list[0][:n_train_sample_per_class, :]
Y_train = np.zeros((X_train.shape[0]*n_class,), dtype=int)

for i in range(n_class-1):
    X_train = np.concatenate((X_train, x_train_list[i+1][:n_train_sample_per_class, :]), axis=0)
    Y_train[(i+1)*n_train_sample_per_class:(i+2)*n_train_sample_per_class] = i+1

X_train.shape, Y_train.shape

((400, 729), (400,))

In [43]:
n_test_sample_per_class = int(2.5*n_train_sample_per_class)

X_test = x_test_list[0][:n_test_sample_per_class, :]
Y_test = np.zeros((X_test.shape[0]*n_class,), dtype=int)

for i in range(n_class-1):
    X_test = np.concatenate((X_test, x_test_list[i+1][:n_test_sample_per_class, :]), axis=0)
    Y_test[(i+1)*n_test_sample_per_class:(i+2)*n_test_sample_per_class] = i+1

X_test.shape, Y_test.shape

((1000, 729), (1000,))

# Dataset Preprocessing

In [44]:
X_train = X_train.reshape(X_train.shape[0], 27, 27, 1)
X_test = X_test.reshape(X_test.shape[0], 27, 27, 1)

X_train.shape, X_test.shape

((400, 27, 27, 1), (1000, 27, 27, 1))

In [45]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

# for i in range(n_class):
#     Y_train[Y_train[:, i] == 1.] = class_label[i]

# for i in range(n_class):
#     Y_test[Y_test[:, i] == 1.] = class_label[i]

# Classical

In [46]:
# Set a random seed
np.random.seed(2020)

In [47]:
import keras.backend as K

# def custom_loss(y_true, y_pred):
#     return K.sum(((y_true.shape[1]-2)*y_true+1)*K.square(y_true-y_pred))/len(y_true)

def custom_loss(y_true, y_pred):
    return K.sum(K.square(y_true-y_pred))/len(y_true)

In [48]:
from tensorflow.keras.layers import *

# define cnn model
def define_model(num_class):
    model = tf.keras.Sequential()
    model.add(Conv2D(2, (3, 3), strides=[2,2], activation='relu', input_shape=(27, 27, 1), use_bias=False))
    #model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(1, (3, 3), strides=[2,2], activation='relu', use_bias=False))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    
    #model.add(Dense(11, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(num_class, activation='softmax'))
    # compile model
    opt = tf.keras.optimizers.Adam(lr=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [49]:
model = define_model(2)

In [50]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 13, 13, 2)         18        
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 6, 6, 1)           18        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 1)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 20        
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________


In [51]:
model(X_train[0:5])

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.6455654 , 0.35443458],
       [0.6759399 , 0.32406   ],
       [0.70199245, 0.29800758],
       [0.6963152 , 0.3036847 ],
       [0.73567104, 0.26432902]], dtype=float32)>

In [52]:
model.fit(X_train, Y_train, epochs=6, batch_size=32, validation_data=(X_test, Y_test), verbose=1)

Epoch 1/6
13/13 [==============================] - 0s 19ms/step - loss: 0.6105 - accuracy: 0.5281 - val_loss: 0.4699 - val_accuracy: 0.6530
Epoch 2/6
13/13 [==============================] - 0s 7ms/step - loss: 0.3924 - accuracy: 0.8290 - val_loss: 0.1878 - val_accuracy: 0.9880
Epoch 3/6
13/13 [==============================] - 0s 8ms/step - loss: 0.1337 - accuracy: 0.9814 - val_loss: 0.0481 - val_accuracy: 0.9900
Epoch 4/6
13/13 [==============================] - 0s 8ms/step - loss: 0.0515 - accuracy: 0.9800 - val_loss: 0.0345 - val_accuracy: 0.9900
Epoch 5/6
13/13 [==============================] - 0s 8ms/step - loss: 0.0393 - accuracy: 0.9881 - val_loss: 0.0302 - val_accuracy: 0.9920
Epoch 6/6
13/13 [==============================] - 0s 8ms/step - loss: 0.0216 - accuracy: 0.9955 - val_loss: 0.0287 - val_accuracy: 0.9930


In [53]:
model.weights

[<tf.Variable 'conv2d_12/kernel:0' shape=(3, 3, 1, 2) dtype=float32, numpy=
 array([[[[ 0.54904276, -0.18866661]],
 
         [[ 0.07036949, -0.35619575]],
 
         [[ 0.73935664, -0.38766655]]],
 
 
        [[[-0.11782569, -0.02037874]],
 
         [[ 0.31866044, -0.3157606 ]],
 
         [[ 0.3749166 ,  0.34303758]]],
 
 
        [[[ 0.501462  , -0.3481154 ]],
 
         [[ 0.4761085 , -0.58461577]],
 
         [[ 0.4933901 , -0.04803445]]]], dtype=float32)>,
 <tf.Variable 'conv2d_13/kernel:0' shape=(3, 3, 2, 1) dtype=float32, numpy=
 array([[[[ 0.08625557],
          [-0.2218078 ]],
 
         [[ 0.10877419],
          [-0.18400016]],
 
         [[ 0.65517133],
          [ 0.10110329]]],
 
 
        [[[-0.5091563 ],
          [-0.32933772]],
 
         [[ 0.08817638],
          [ 0.31833714]],
 
         [[ 0.65596557],
          [-0.6361094 ]]],
 
 
        [[[ 0.08975314],
          [-0.07224308]],
 
         [[ 0.7016689 ],
          [-0.47376275]],
 
         [[ 0.41581964],
 

In [239]:
predict_test = model.predict(X_test)